In [1]:
import spacy 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import unicodedata
import inflect,gensim
from gensim import corpora
from gensim.models import LsiModel
nlp = spacy.load("en_core_web_sm")

In [2]:
def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    new_words = []
    for word in words:
        if word not in stop_words and len(word)>2:
            new_words.append(word)
    return new_words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return words

In [3]:
def getTokens(Text):
    coumpound_nlp = nlp(Text)

    coumpound_word = []
    for X in coumpound_nlp.ents:
        s = re.sub(r'[^\w\s]','',X.text.lower())
        s = re.sub("[\s.;:,*]+", " ", s)
        coumpound_word.append(s)

    tokens = word_tokenize(Text.lower())
    fTokens = []
    word = ''
    for t in tokens:
        if word == '':
            word = t
        else:
            word += ' '+t
        find = False
        for X in coumpound_word:
            if word in X:
                find = True
                if word == X:
                    fTokens.append(word)
                    word = ''
                    break
        if find == False:
            tokensW = word_tokenize(word)
            for w in tokensW:
                fTokens.append(w)
            word = ''
    fTokens = normalize(fTokens)
    return fTokens

In [4]:
from nltk.corpus import words as engwords
def getTokens_withoutNames(Text):
    coumpound_nlp = nlp(Text)

    tokens = normalize(word_tokenize(Text.lower()))
    fTokens = []
    word = ''
    for t in tokens:
        find = False
        for X in coumpound_nlp.ents:
            if t in X.text.lower():
                if X.label_ != 'PERSON':
                    find = True
                    break;
        if find == False:
            if t in engwords.words():
                fTokens.append(t)
    return fTokens

In [5]:
import math
from textblob import TextBlob as tb

def tf(word, words):
    return words.count(word) / len(words)

def n_containing(word, wordslist):
    return sum(1 for words in wordslist if word in words)

def idf(word, wordslist):
    return math.log(len(wordslist) / (1 + n_containing(word, wordslist)))

def tfidf(word, words, wordslist):
    return tf(word, words) * idf(word, wordslist)

In [6]:
def traitement1(line):
    words = getTokens(line[1])
    print("Top interest :")
    scores = {word: tf(word, words) for word in words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF: {}".format(word, round(score, 5)))

    

In [7]:
def traitement(line):
    
    words = getTokens(line[1])
    print("Top interest :")
    scores = {word: tf(word, words) for word in words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF: {}".format(word, round(score, 5)))
    
    line2 = nlp(line[2])
    print("University :")
    for X in line2.ents:
        if 'University' in X.text: 
            print("\t"+X.text.replace('\n',' '))
    
    print("Advisors :")
    for X in line2.ents:
        if X.label_ == 'PERSON':
            print("\t"+X.text.replace('\n',' '))
    
    print("laboratories or research teams")
    line3 = nlp(line[3])
    for X in line3.ents:
        if X.label_ == 'ORG':
            print("\t"+X.text.replace('\n',' '))
    
    print("collaborations")
    line4 = nlp(line[4])
    collaborations = []
    for X in line4.ents:
        if X.label_ == 'PERSON':
            find = False
            for c in collaborations:
                if X.text in c:
                    find = True
            if find == False:
                collaborations.append(X.text.replace("[\s]+",' '))

    for c in collaborations:
        print("\t"+c)
    
    

In [8]:

import csv
with open('CVs.csv', 'r', encoding="utf-8") as f:
    theReader = csv.reader(f)
    processed_words = [] 
    publications = []
    done = False
    for line in theReader:
        if done:
            print("\n"+line[0])
            lwords = traitement(line)
            publications.append(line[4])
        else:
            done = True


 Lu Jiang 
Top interest :
	Word: research, TF: 0.06667
	Word: big data, TF: 0.06667
	Word: goal, TF: 0.03333
University :
	Carnegie Mellon University
	Free University of Brussels
	Xi’an Jiaotong University
Advisors :
	Alexander Hauptmann
	Teruko Mitamura
	Multimedia Search
	Rank
	Jun Liu B.Eng
	GPA
laboratories or research teams
	Google AI Cloud
	Carnegie Mellon University
	IARPA
	NIST TRECVID
	Yahoo Research
	Google Research
	YouTube
	Microsoft
	Xi’an Jiaotong University
	PDF
	HTML
collaborations
	Lu Jiang
	Zhengyuan Zhou
	Thomas Leung
	Li-Jia Li
	Li Fei-Fei
	Junwei Liang
	Liangliang Cao
	Alexander Hauptmann
	Zelun Luo
	Jun-Ting Hsieh
	Juan Carlos Niebles
	Yu Wu
	Linchao Zhu
	Yi Yang
	Decoupled Novel
	Yannis Kalantidis
	Jiliang Tang
	Alex Hauptmann
	Deyu Meng
	Prior Knowledge
	ICMR
	Junwei Han
	Senmao Ye
	Xiaojun Chang
	Multimedia Search
	Teruko Mitamura
	Videos
	Qian Zhao
	Qi Xie
	Zongben Xu
	Dingwen Zhang
	Li Chao
	Zhao Qian
	Shoou
	Zhen-Zhong

JUAN GARCIA 
Top interest :
	Word: in

In [9]:
processed_docs = []

for doc in publications:
    processed_docs.append(getTokens_withoutNames(doc))

In [10]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [11]:
#dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [12]:
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 7, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [13]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.043*"cutler" + 0.043*"cell" + 0.043*"professional" + 0.023*"noise" + 0.023*"constrained" + 0.023*"capable" + 0.023*"human" + 0.023*"antibody" + 0.023*"conformation" + 0.023*"immunogenic"


Topic: 1 
Words: 0.105*"conservation" + 0.072*"economist" + 0.038*"model" + 0.038*"south" + 0.038*"analysis" + 0.038*"towards" + 0.038*"economy" + 0.038*"environment" + 0.005*"professional" + 0.005*"entry"


Topic: 2 
Words: 0.096*"title" + 0.073*"year" + 0.050*"issue" + 0.050*"produced" + 0.050*"manuscript" + 0.050*"page" + 0.050*"volume" + 0.027*"soft" + 0.027*"oppositely" + 0.027*"matter"


Topic: 3 
Words: 0.051*"liang" + 0.026*"oral" + 0.026*"artificial" + 0.026*"detection" + 0.014*"optical" + 0.014*"portrait" + 0.014*"power" + 0.014*"intensity" + 0.014*"parle" + 0.014*"capacity"


Topic: 4 
Words: 0.081*"chromatin" + 0.042*"cationic" + 0.042*"histone" + 0.022*"terminus" + 0.022*"influence" + 0.022*"domain" + 0.022*"winged" + 0.022*"amino" + 0.022*"organization" + 0.022*"yan"
